## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

#fix bug : collections has no attribute iterable
import collections
collections.Iterable = collections.abc.Iterable

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,70.88,56,94,6.82,overcast clouds
1,1,Batticaloa,LK,7.7102,81.6924,82.44,74,100,4.54,moderate rain
2,2,Labuhan,ID,-6.8844,112.2051,79.25,77,55,9.06,broken clouds
3,3,Bengkulu,ID,-3.8004,102.2655,79.16,83,87,2.75,overcast clouds
4,4,San Andres,CO,12.5847,-81.7006,84.22,79,40,9.22,scattered clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Input preferred minimum temperature for your travel:"))
max_temp = float(input("Input preferred maximum temperature for your travel:"))

In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
eligible_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
eligible_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Srednekolymsk,RU,67.4500,153.6833,43.86,72,8,12.44,clear sky
6,6,Punta Arenas,CL,-53.1500,-70.9167,41.11,87,100,23.02,moderate rain
21,21,Ushuaia,AR,-54.8000,-68.3000,44.26,65,75,9.22,broken clouds
24,24,New Norfolk,AU,-42.7826,147.0587,44.10,82,68,1.01,broken clouds
28,28,Paamiut,GL,61.9940,-49.6678,41.49,91,26,19.10,scattered clouds
38,38,Barrow,US,71.2906,-156.7887,33.06,97,100,12.66,overcast clouds
48,48,Dunedin,NZ,-45.8742,170.5036,43.81,79,100,13.00,overcast clouds
56,56,Qaanaaq,GL,77.4840,-69.3632,38.68,72,12,0.85,few clouds
65,65,Longyearbyen,SJ,78.2186,15.6401,42.64,75,40,20.71,scattered clouds
70,70,Mount Gambier,AU,-37.8333,140.7667,42.26,89,19,2.64,few clouds


In [18]:
# 4a. Determine if there are any empty rows.
eligible_cities_df.count()

Unnamed: 0             40
City                   40
Country                40
Lat                    40
Lng                    40
Max Temp               40
Humidity               40
Cloudiness             40
Wind Speed             40
Current Description    40
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = eligible_cities_df.dropna()
clean_df.count()

Unnamed: 0             40
City                   40
Country                40
Lat                    40
Lng                    40
Max Temp               40
Humidity               40
Cloudiness             40
Wind Speed             40
Current Description    40
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Srednekolymsk,RU,43.86,clear sky,67.4500,153.6833,
6,Punta Arenas,CL,41.11,moderate rain,-53.1500,-70.9167,
21,Ushuaia,AR,44.26,broken clouds,-54.8000,-68.3000,
24,New Norfolk,AU,44.10,broken clouds,-42.7826,147.0587,
28,Paamiut,GL,41.49,scattered clouds,61.9940,-49.6678,
38,Barrow,US,33.06,overcast clouds,71.2906,-156.7887,
48,Dunedin,NZ,43.81,overcast clouds,-45.8742,170.5036,
56,Qaanaaq,GL,38.68,few clouds,77.4840,-69.3632,
65,Longyearbyen,SJ,42.64,scattered clouds,78.2186,15.6401,
70,Mount Gambier,AU,42.26,few clouds,-37.8333,140.7667,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping.....")

hotel_df.head(10)
        

Hotel not found. Skipping.....
Hotel not found. Skipping.....
Hotel not found. Skipping.....
Hotel not found. Skipping.....


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Srednekolymsk,RU,43.86,clear sky,67.4500,153.6833,
6,Punta Arenas,CL,41.11,moderate rain,-53.1500,-70.9167,Hotel Hain
21,Ushuaia,AR,44.26,broken clouds,-54.8000,-68.3000,Albatross Hotel
24,New Norfolk,AU,44.10,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages
28,Paamiut,GL,41.49,scattered clouds,61.9940,-49.6678,Pistut
38,Barrow,US,33.06,overcast clouds,71.2906,-156.7887,King Eider Inn
48,Dunedin,NZ,43.81,overcast clouds,-45.8742,170.5036,Scenic Hotel Southern Cross
56,Qaanaaq,GL,38.68,few clouds,77.4840,-69.3632,Qaanaaq Hotel
65,Longyearbyen,SJ,42.64,scattered clouds,78.2186,15.6401,"Radisson Blu Polar Hotel, Spitsbergen"
70,Mount Gambier,AU,42.26,few clouds,-37.8333,140.7667,The Commodore


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Srednekolymsk,RU,43.86,clear sky,67.4500,153.6833,
6,Punta Arenas,CL,41.11,moderate rain,-53.1500,-70.9167,Hotel Hain
21,Ushuaia,AR,44.26,broken clouds,-54.8000,-68.3000,Albatross Hotel
24,New Norfolk,AU,44.10,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages
28,Paamiut,GL,41.49,scattered clouds,61.9940,-49.6678,Pistut


In [23]:
# 8a. Create the output File (CSV)
output_file = ('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

pandas.core.frame.DataFrame

In [28]:
# 11a. Add a marker layer for each city to the map.
current_description = clean_hotel_df["Current Description"]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))